In [1]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection, Trainer
import torch
import sys
import os
import pandas as pd
from pycocotools.coco import COCO
from tqdm import tqdm
from functools import partial

sys.path.append("../../../transformers/Dongjin/1011_model_search")
import utils
import dataset
import train_eval


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '../../../transformers/Dongjin/1011_model_search/result/1015/jozhang97/deta-swin-large_3_img_size_720'
coco_dir_path = '../../../Split_data'
checkpoint_dir = 'checkpoint-15704'
device = 'cuda'

checkpoint_path = os.path.join(model_path, checkpoint_dir)
run_name = os.path.split(model_path)[-1]
json_path = os.path.join(model_path, run_name + '.json')

conf = utils.read_json(json_path)
conf['coco_dir_path'] = coco_dir_path
train_info_path = os.path.join(conf['coco_dir_path'], conf['train_info_name'])
valid_info_path = os.path.join(conf['coco_dir_path'], conf['valid_info_name'])

coco_train = COCO(train_info_path)
coco_valid = COCO(valid_info_path)

id2label = utils.get_id2label(conf['classes'])
label2id = utils.get_label2id(id2label)

train = dataset.COCO2dataset(conf['data_dir_path'], coco_train)
valid = dataset.COCO2dataset(conf['data_dir_path'], coco_valid)

image_processor = AutoImageProcessor.from_pretrained(checkpoint_path)
model = AutoModelForObjectDetection.from_pretrained(checkpoint_path)
model.to(device)


loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


DetaForObjectDetection(
  (model): DetaModel(
    (backbone): DetaBackboneWithPositionalEncodings(
      (model): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0): SwinLayer(
                  (layernorm_before): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=192, out_features=192, bias=True)
                      (key): Linear(in_features=192, out_features=192, bias=True)
                      (value): Linear(in_features=1

In [3]:
import numpy as np

def test_eval(conf, model, image_processor, coco_data, data, result_path):
    batch_size = 8

    image_names = []
    prediction_strings = []
    batch_indices = get_batch_indices(batch_size, len(data))

    for batch_index in tqdm(batch_indices):
        batch = data[batch_index]
        image_name, prediction_string = get_predictions(batch, coco_data, model, image_processor)
        
        image_names.extend(image_name)
        prediction_strings.extend(prediction_string)

    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = image_names
    submission.to_csv(result_path, index=None)


def get_batch_indices(batch_size, n):
    i = 0
    batch_index = []
    batch_indices = []

    while True:
        if (i == n): 
            batch_indices.append(batch_index)
            break

        batch_index.append(i)

        if (len(batch_index) % batch_size == 0):
            batch_indices.append(batch_index)
            batch_index = []
            
        i += 1  

    return batch_indices
        

def get_predictions(batch, coco_data, model, image_processor):
    threshold = 0.05
    device = "cuda"

    with torch.no_grad():
        images = [np.array(image) for image in batch['image']]
        images = [np.fliplr(image) for image in images] # horizontal flip
        image_ids = batch['image_id']
        image_infos = coco_data.loadImgs(image_ids)

        inputs = image_processor(images=images, return_tensors="pt")
        outputs = model(**inputs.to(device))
        target_sizes = [image.shape for image in images]
        results = image_processor.post_process_object_detection(outputs, threshold=threshold, target_sizes=target_sizes)
        
        # horizontal flip
        for taget_size, result in zip(target_sizes, results):
            x1 = result['boxes'][:, 0]
            x2 = result['boxes'][:, 2]
            w, h, _ = taget_size
            x1, x2 = w - x2, w - x1
            result['boxes'][:, 0] = x1
            result['boxes'][:, 2] = x2


        image_names = [image_info['file_name'] for image_info in image_infos]
        prediction_strings = []

        for result in results:
            _, indices = torch.sort(result['scores'], descending=True)
            result["scores"] = result["scores"][indices].detach().cpu().numpy() 
            result["labels"] = result["labels"][indices].detach().cpu().numpy() 
            result["boxes"] = result["boxes"][indices].detach().cpu().numpy() 

            prediction_string = ''

            for box, score, label in zip(result["boxes"], result["scores"], result["labels"]):
                prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                        box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '

            prediction_strings.append(prediction_string)
        
        return image_names, prediction_strings

In [4]:
os.makedirs('result', exist_ok=True)
test_eval(conf, model, image_processor, coco_valid, valid, 'result/' + run_name + '_valid.csv')


  0%|          | 0/120 [00:00<?, ?it/s]

{'scores': tensor([0.7441, 0.7353, 0.7322, 0.7318, 0.7002, 0.6796, 0.6282, 0.6057, 0.3084,
        0.2251, 0.1991, 0.1882, 0.1861, 0.1757, 0.1583, 0.1578, 0.1543, 0.1511,
        0.1491, 0.1488, 0.1477, 0.1471, 0.1466, 0.1459, 0.1454, 0.1419, 0.1417,
        0.1336, 0.1331, 0.1330, 0.1297, 0.1250, 0.1243, 0.1173, 0.1147, 0.1130,
        0.1125, 0.1119, 0.1113, 0.1062, 0.1059, 0.1056, 0.1035, 0.1030, 0.1024,
        0.1023, 0.1021, 0.0990, 0.0990, 0.0982, 0.0979, 0.0971, 0.0954, 0.0947,
        0.0931, 0.0923, 0.0920, 0.0883, 0.0880, 0.0879, 0.0875, 0.0873, 0.0854,
        0.0850, 0.0845, 0.0845, 0.0843, 0.0839, 0.0834, 0.0829, 0.0824, 0.0818,
        0.0815, 0.0799, 0.0786, 0.0784, 0.0780, 0.0760, 0.0749, 0.0746, 0.0733,
        0.0730, 0.0722, 0.0721, 0.0716, 0.0708, 0.0681, 0.0664, 0.0663, 0.0656,
        0.0652, 0.0648, 0.0642, 0.0640, 0.0639, 0.0638, 0.0638, 0.0638, 0.0635,
        0.0630], device='cuda:0'), 'labels': tensor([1, 3, 0, 0, 5, 5, 3, 2, 5, 3, 5, 4, 3, 1, 5, 3, 5, 4, 2,

In [7]:
images

NameError: name 'images' is not defined